In [1]:
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.notebook import tqdm
import torchvision.models as models
from model import *
from utils import *

In [2]:
transform = transforms.Compose([
    transforms.ToPILImage(),  
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

def transform_triplet(triplet_list, batch_size=256, apply_transform=True):
    batch_steps = len(triplet_list) // batch_size
    
    for i in range(batch_steps + 1):
        anchor, positive, negative = [], [], []

        j = i * batch_size
        while j < (i + 1) * batch_size and j < len(triplet_list):
            a, p, n = triplet_list[j]
            a_img = read_image(a)
            p_img = read_image(p)
            n_img = read_image(n)

            if a_img is not None and p_img is not None and n_img is not None:
                if apply_transform:
                    a_img = transform(a_img)
                    p_img = transform(p_img)
                    n_img = transform(n_img)

                anchor.append(a_img)
                positive.append(p_img)
                negative.append(n_img)
            j += 1

        if not anchor or not positive or not negative:
            continue  

        
        anchor = torch.stack(anchor)
        positive = torch.stack(positive)
        negative = torch.stack(negative)

        yield ([anchor, positive, negative])

In [3]:
random.seed(5)
np.random.seed(5)


ROOT = "Extracted Faces"

train_list, test_list = train_test_split(ROOT, split=0.8)
print("Length of training list:", len(train_list))
print("Length of testing list :", len(test_list))
print("\nTest List:", test_list)

Length of training list: 1062
Length of testing list : 266

Test List: {'1514': 2, '1604': 3, '1640': 13, '966': 2, '326': 2, '1016': 6, '440': 2, '1084': 3, '492': 2, '459': 2, '1122': 2, '1408': 2, '1676': 9, '553': 8, '65': 2, '797': 3, '732': 4, '0': 4, '1138': 3, '1218': 2, '1232': 2, '1436': 4, '1180': 3, '419': 2, '502': 2, '604': 3, '807': 5, '803': 4, '1074': 4, '1185': 2, '1183': 3, '337': 3, '1070': 2, '722': 4, '684': 2, '1492': 2, '1163': 3, '233': 2, '317': 2, '1340': 2, '281': 4, '851': 16, '533': 2, '120': 2, '545': 3, '184': 3, '696': 2, '1553': 2, '1313': 3, '1652': 21, '1671': 4, '1526': 3, '1097': 6, '293': 2, '1024': 8, '1398': 6, '1380': 6, '1150': 8, '177': 2, '563': 2, '623': 2, '1137': 4, '1260': 2, '1266': 4, '1099': 3, '744': 2, '932': 2, '96': 3, '630': 2, '26': 2, '473': 7, '1418': 2, '361': 3, '328': 3, '897': 7, '1519': 2, '1263': 2, '1164': 2, '1674': 2, '982': 2, '785': 3, '460': 3, '520': 9, '743': 2, '335': 2, '1413': 3, '1461': 4, '1267': 4, '1606': 

In [4]:
train_triplet = generate_triplet(ROOT, train_list)
test_triplet  = generate_triplet(ROOT, test_list)

print("Number of training triplets:", len(train_triplet))
print("Number of testing triplets :", len(test_triplet))

print("\nExamples of triplets:")
for i in range(5):
    print(train_triplet[i])

Number of training triplets: 9074
Number of testing triplets : 2218

Examples of triplets:
(('571', '5.jpg'), ('571', '6.jpg'), ('1282', '1.jpg'))
(('841', '5.jpg'), ('841', '9.jpg'), ('1662', '3.jpg'))
(('536', '0.jpg'), ('536', '6.jpg'), ('865', '0.jpg'))
(('38', '5.jpg'), ('38', '9.jpg'), ('1511', '1.jpg'))
(('1098', '4.jpg'), ('1098', '5.jpg'), ('366', '3.jpg'))


In [7]:
encoder = load_pretrained_model(input_shape=(3, 128, 128))
siamese_network = SiameseNetwork(encoder)

/Users/alexisrevelle/miniconda3/envs/data-science/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alexisrevelle/miniconda3/envs/data-science/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
siamese_model = SiameseModel(siamese_network)
optimizer = optim.Adam(siamese_model.parameters(), lr=1e-3, eps=1e-01)

def triplet_loss(ap_distance, an_distance):
    loss = ap_distance - an_distance + 0.5
    loss = torch.max(loss, torch.zeros_like(loss))  
    return loss.mean()

In [9]:
epochs = 10
batch_size = 100
train_loss = []
test_metrics = []


for epoch in tqdm(range(1, epochs + 1)):
    t = time.time()
    siamese_model.train()
    epoch_loss = []

    for data in transform_triplet(train_triplet, batch_size=batch_size):
        optimizer.zero_grad()
        anchor, positive, negative = data  
        ap_distance, an_distance = siamese_model(anchor, positive, negative)
        loss = triplet_loss(ap_distance, an_distance)   
        loss.backward()  
        optimizer.step() 
        epoch_loss.append(loss.item())

    epoch_loss = sum(epoch_loss) / len(epoch_loss)
    train_loss.append(epoch_loss)

    print(f"\nEPOCH: {epoch} \t (Epoch done in {int(time.time()-t)} sec)")
    print(f"Loss on train    = {epoch_loss:.5f}")

    
    metric = accuracy_model(siamese_model, test_triplet, batch_size=batch_size)
    test_metrics.append(metric)
    accuracy = metric[0]


torch.save(siamese_model.state_dict(), "siamese_model-final")

  0%|          | 0/10 [00:00<?, ?it/s]


EPOCH: 1 	 (Epoch done in 3209 sec)
Loss on train    = 0.12770
Accuracy on test = 0.72858

EPOCH: 2 	 (Epoch done in 3289 sec)
Loss on train    = 0.05792
Accuracy on test = 0.75924

EPOCH: 3 	 (Epoch done in 3195 sec)
Loss on train    = 0.02653
Accuracy on test = 0.77277

EPOCH: 4 	 (Epoch done in 3099 sec)
Loss on train    = 0.01113
Accuracy on test = 0.76150

EPOCH: 5 	 (Epoch done in 3096 sec)
Loss on train    = 0.00425
Accuracy on test = 0.76240

EPOCH: 6 	 (Epoch done in 3097 sec)
Loss on train    = 0.00145
Accuracy on test = 0.76105

EPOCH: 7 	 (Epoch done in 3096 sec)
Loss on train    = 0.00045
Accuracy on test = 0.76285

EPOCH: 8 	 (Epoch done in 3088 sec)
Loss on train    = 0.00013
Accuracy on test = 0.76510

EPOCH: 9 	 (Epoch done in 3116 sec)
Loss on train    = 0.00005
Accuracy on test = 0.76060

EPOCH: 10 	 (Epoch done in 3082 sec)
Loss on train    = 0.00002
Accuracy on test = 0.75969


In [1]:
accuracies = [item[0] for item in test_metrics]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# plot 1 train loss
ax1.plot(train_loss, label='Train Loss')
ax1.set_title('Model Train Loss Over Epochs')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Train loss')
ax1.set_xticks(range(len(train_loss)))
ax1.set_xticklabels(range(1, len(train_loss) + 1))
ax1.legend()
ax1.grid(True)

# plot 2 accuracy on test
ax2.plot(accuracies, label='Accuracy')
ax2.set_title('Model Accuracy on Testset Over Epochs')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.set_xticks(range(len(accuracies)))
ax2.set_xticklabels(range(1, len(accuracies) + 1))
ax2.legend()
ax2.grid(True)

# Display the plots
plt.show()

NameError: name 'test_metrics' is not defined